In [ ]:
!pip install nbimporter

In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# import torch.nn.init as init
from torch.autograd import Variable
from dataLoader import loadData
from tensorflow import keras

In [2]:
# # Downloading Movielens-1m
# !curl -O http://files.grouplens.org/datasets/movielens/ml-1m.zip
# #     http://www.grouplens.org/system/files/ml-1m.zip
# !unzip ml-1m.zip
# !cd ml-1m/

In [3]:
# import zipfile
# with zipfile.ZipFile('ml-1m.zip', 'r') as zip_ref:
#     zip_ref.extractall('./')

In [4]:
seed = 47
np.random.seed(seed)

In [5]:
# load data
tr, vr = loadData('./ml-1m/ratings.dat', delimiter='::', seed=seed, transpose=False, valfrac=0.1)

reading data...
data read in 5.49533748626709 seconds
loaded dense data matrix


In [6]:
np.save('./tr_movielens_1m', tr)
np.save('./vr_movielens_1m', vr)

In [7]:
tr

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
vr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
def get_sparsity(mat):
    sparsity = float(len(mat.nonzero()[0]))
    sparsity /= (mat.shape[0] * mat.shape[1])
    sparsity *= 100
    return sparsity

In [10]:
get_sparsity(tr)

4.021525859265269

In [11]:
get_sparsity(vr)

0.44683670296601535

In [12]:
train = tr.copy()

In [13]:
if torch.cuda.is_available() == True:
    device = "cuda"
else:
    device = "cpu"

# Training Denoising Autoencoder

In [14]:
############## Pytorch model doesn't converge - to do - check #################

from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import regularizers

def autoEncoder(X):
    '''
    Autoencoder for Collaborative Filter Model
    '''

    # Input
    input_layer = Input(shape=(X.shape[1],), name='UserScore')
    
    # Encoder
    # -----------------------------
    enc = Dense(512, activation='selu', name='EncLayer1', kernel_regularizer=regularizers.l2(0.000001))(input_layer)

    # Latent Space
    # -----------------------------
    lat_space = Dense(512, activation='selu', name='LatentSpace', kernel_regularizer=regularizers.l2(0.000001))(enc)
    lat_space = Dropout(0.5, name='Dropout')(lat_space) # Dropout

    # Decoder
    # -----------------------------
    dec = Dense(512, activation='selu', name='DecLayer1', kernel_regularizer=regularizers.l2(0.000001))(lat_space)

    # Output
    output_layer = Dense(X.shape[1], activation='linear', name='UserScorePred', kernel_regularizer=regularizers.l2(0.000001))(dec)

    # this model maps an input to its reconstruction
    model = Model(input_layer, output_layer)    
    
    return model

Using TensorFlow backend.


In [15]:
zero_mask = (train == 0)
positive_feedback_mask = (train > 3)
negative_feedback_mask = ((train < 4) * (1 - zero_mask))

In [16]:
positive_feedback_mask.sum(), negative_feedback_mask.sum()

(517621, 382567)

In [17]:
assert (positive_feedback_mask + negative_feedback_mask != zero_mask).all()
assert (positive_feedback_mask + negative_feedback_mask == 1 - zero_mask).all()

In [18]:
get_sparsity(zero_mask), get_sparsity(positive_feedback_mask), get_sparsity(negative_feedback_mask)

(95.97847414073473, 2.3124349989099473, 1.7090908603553212)

In [19]:
100 - get_sparsity(zero_mask), get_sparsity(positive_feedback_mask), get_sparsity(negative_feedback_mask), get_sparsity(positive_feedback_mask) + get_sparsity(negative_feedback_mask)

(4.021525859265267, 2.3124349989099473, 1.7090908603553212, 4.021525859265268)

In [20]:
P = 0.4
mask_arr_neg = (np.random.rand(negative_feedback_mask.shape[0], negative_feedback_mask.shape[1]) > P)
y_neg = negative_feedback_mask
X_neg = negative_feedback_mask*mask_arr_neg # corrupting 

In [21]:
mask_arr_pos = (np.random.rand(positive_feedback_mask.shape[0], positive_feedback_mask.shape[1]) > P)
y_pos = positive_feedback_mask
X_pos = positive_feedback_mask*mask_arr_pos # corrupting 

In [22]:
corrupted_tr = (mask_arr_neg*(tr>0)).astype(int)

In [23]:
get_sparsity(tr), get_sparsity(X_neg), get_sparsity(y_neg), get_sparsity(X_pos), get_sparsity(y_pos), get_sparsity(corrupted_tr)

(4.021525859265269,
 1.026195215919772,
 1.7090908603553212,
 1.3870741200058612,
 2.3124349989099473,
 2.4134703702247653)

In [24]:
model_neg = autoEncoder(X_neg)
model_neg.compile(optimizer = Adam(lr=0.0001), loss='mse')

model_pos = autoEncoder(X_pos)
model_pos.compile(optimizer = Adam(lr=0.0001), loss='mse')

In [25]:
model_neg = keras.models.load_model('./model_neg')
model_pos = keras.models.load_model('./model_pos')

In [26]:
model_neg.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
UserScore (InputLayer)       [(None, 3706)]            0         
_________________________________________________________________
EncLayer1 (Dense)            (None, 512)               1897984   
_________________________________________________________________
LatentSpace (Dense)          (None, 512)               262656    
_________________________________________________________________
Dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
DecLayer1 (Dense)            (None, 512)               262656    
_________________________________________________________________
UserScorePred (Dense)        (None, 3706)              1901178   
Total params: 4,324,474
Trainable params: 4,324,474
Non-trainable params: 0
_________________________________________________

In [ ]:
# predicted_tr = np.load('predicted_tr.npy')
# augmented_train = np.load('augmented_train.npy')

In [ ]:
hist_neg = model_neg.fit(x=X_neg, y=y_neg,
                  epochs=300,
                  batch_size=128,
                  shuffle=True,
# augmented_train = np.load('augmented_train.npy')
                  validation_split=0.0)

In [ ]:
hist_pos = model_pos.fit(x=X_pos, y=y_pos,
                  epochs=300,
                  batch_size=128,
                  shuffle=True,
                  validation_split=0.0)

In [ ]:
import matplotlib.pyplot as plt 

def plot_hist(hist):
    # summarize history for loss
    fig, ax = plt.subplots()  # create figure & 1 axis

    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    plt.plot(hist.history['loss'])
    #plt.plot(hist.history['val_loss'])

In [ ]:
plot_hist(hist_neg)

In [ ]:
plot_hist(hist_pos)

In [ ]:
model_neg.save('./model_neg')
model_pos.save('./model_pos')

In [27]:
np.set_printoptions(precision=3)
predicted_neg = model_neg.predict(X_neg)
predicted_pos = model_pos.predict(X_pos)

In [28]:
(predicted_neg > 0.4).sum(), (y_neg == 1).sum() # predicted vs real

(170143, 382567)

In [29]:
(predicted_neg>0.5).sum(), (predicted_pos>0.5).sum() # trained on

(115775, 214496)

In [30]:
print((y_neg * (predicted_neg>0.4)).sum()/(predicted_neg>0.4).sum()) # accuracy on actual 
print((y_pos * (predicted_pos>0.4)).sum()/(predicted_pos>0.4).sum())

0.9528337927508037
0.9187441580985445


In [32]:
print((y_pos * (predicted_neg>0.4)).sum()/y_pos.sum()) # just to see that it's a low number
print((y_neg * (predicted_pos>0.4)).sum()/y_neg.sum())

0.004936043939484681
0.016499070751005707


In [33]:
(y_neg * (predicted_neg>0.5)).sum()/((predicted_neg>0.5).sum())

0.9862405527963722

In [34]:
(y_pos * (predicted_pos>0.5)).sum()/((predicted_pos>0.5).sum())

0.9698642398925854

In [35]:
((predicted_neg>0.5)  * (X_neg<0.5)).sum() # predicted values which were not in the train matrix

39637

In [36]:
((predicted_pos>0.5)  * (X_pos<0.5)).sum() # predicted values which were not in the train matrix

69892

In [37]:
((y_neg * (((predicted_neg>0.5)  * (X_neg<0.5)))) == 1).sum()/(((predicted_neg>0.5)  * (X_neg<0.5))).sum()

0.9598102782753488

In [38]:
((y_pos * (((predicted_pos>0.5)  * (X_pos<0.5)))) == 1).sum()/(((predicted_pos>0.5)  * (X_pos<0.5))).sum()

0.9075144508670521

In [39]:
to_add_negative = model_neg.predict(y_neg)
to_add_positive = model_neg.predict(y_pos)

In [40]:
to_add_negative = (to_add_negative>0.5).astype(int)
to_add_positive = (to_add_positive>0.5).astype(int)

In [41]:
print((y_pos * to_add_negative).sum()/y_pos.sum()) # That's working
print((y_neg * to_add_positive).sum()/y_neg.sum())

0.01513076169629903
0.04135746156882324


In [91]:
to_add_negative.sum(), to_add_positive.sum()

(207742, 234140)

In [43]:
(y_neg* (to_add_negative>0.8)).sum()/(((to_add_negative>0.8)).sum()), (y_pos* (to_add_positive>0.8)).sum()/(((to_add_positive>0.8)).sum())

(0.8858054702467484, 0.744853506449133)

In [ ]:
# to keep the balance
threshold_neg = 0.2
threshold_pos = 0.3 

In [ ]:
((to_add_negative > threshold_neg) * (tr==0)).sum(), ((to_add_positive > threshold_pos) * (tr==0)).sum() # new values # new values

In [ ]:
p_probs_neg = [(tr == 1).sum()/((tr > 0) & (tr < 4)).sum(), (tr == 2).sum()/(((tr > 0) & (tr < 4))).sum(), (tr == 3).sum()/((tr > 0) & (tr < 4)).sum()]
p_probs_neg

In [ ]:
p_probs_pos = [(tr == 4).sum()/((tr > 3) & (tr <= 5)).sum(), (tr == 5).sum()/((tr > 3) & (tr <= 5)).sum()]
p_probs_pos

In [ ]:
augmented_train = tr + (to_add_negative > threshold_neg) * (tr == 0) * np.random.choice(np.arange(1, 4), tr.shape, p=p_probs_neg) + (to_add_positive > threshold_pos) * (tr == 0) * np.random.choice(np.arange(4, 6), tr.shape, p=p_probs_pos)

In [ ]:
get_sparsity(tr), get_sparsity(augmented_train) # reduced sparsity

In [ ]:
(np.isin(tr, augmented_train)).all()

In [ ]:
print(5, (5 == tr).sum())
print(4, (4 == tr).sum())
print(3, (3 == tr).sum())
print(2, (2 == tr).sum())
print(1, (1 == tr).sum())
print(0, (0 == tr).sum())

In [ ]:
print(5, (5 == augmented_train).sum())
print(4, (4 == augmented_train).sum())
print(3, (3 == augmented_train).sum())
print(2, (2 == augmented_train).sum())
print(1, (1 == augmented_train).sum())
print(0, (0 == augmented_train).sum())

In [ ]:
((tr == 0) * (augmented_train > 0)).sum() # new values

In [ ]:
# np.save('predicted_tr', predicted_tr)
np.save('augmented_train', augmented_train)

In [44]:
features_length = train.shape[1]
class NetD(torch.nn.Module):
    def __init__(self, feat_size):
        super(NetD, self).__init__()
        self.feat_size = feat_size
#         self.use_cuda = True
#         self.feat_size = feat_size
        # top
#         print(self.feat_size*2)
        self.t1 = torch.nn.Linear(self.feat_size, 1024)
        # bottom
        self.b1 = torch.nn.Linear(self.feat_size, 1024)
        # combined
        self.fc = torch.nn.Linear(2 * 1024, self.feat_size)
    def forward(self, xr, xf):
        # get filt
        
        filt = 1 - (xr * (xf >= 0.5).float()) - ((1-xr) * (xf < 0.5).float())
        # random swap
        idr = torch.multinomial(torch.Tensor([0.5,0.5]), xr.size(0), replacement=True)
        idrx = idr.float().unsqueeze(1).expand_as(xr)
#         if self.use_cuda: 
        idrx = idrx.cuda()
        idrx = Variable(idrx)
        xt = xr * idrx + xf * (1 - idrx)
        xb = xr * (1 - idrx) + xf * idrx
        # top : real
        xt = F.relu(self.t1(xt))
        # bottom : fake
        xb = F.relu(self.b1(xb))
        # combined
        x = torch.cat((xt, xb), 1)
        x = F.tanh(self.fc(x))
        # apply filter, aggregate
        x = filt * x
        x = x.mean(dim = 1).squeeze()
        # use sign, because of swapping
        sgn = idr * 2 - 1
        sgn = sgn.cuda()
        sgn = Variable(sgn.float())
        x = sgn * x
        return x


class NetG(nn.Module):
    
    def __init__(self, feat_size):

        super(NetG, self).__init__()
        self.feat_size = feat_size
        self.netGen = torch.nn.Sequential( 
                                torch.nn.Linear(nz + self.feat_size, 1024), 
#                                 torch.nn.BatchNorm1d(1024),
                                torch.nn.ReLU(), 
#                                 nn.Dropout(0.5),
#                                 torch.nn.Linear(1024, 1024),
# #                                 torch.nn.BatchNorm1d(1024),
#                                 torch.nn.ReLU(), 
#                                 nn.Dropout(0.6),
                                torch.nn.Linear(1024, features_length), 
                                torch.nn.Sigmoid()
#                                 torch.nn.BatchNorm1d(features_length),
#                                 nn.Dropout(0.7),
#                                 torch.nn.Sigmoid()
                                )

        
    def forward(self, e_mask, x):
        x = self.netGen(x)
        x = x * e_mask
        return x
#         return F.dropout(x, 0.7)
#         return 5 * self.netGen(x)
#         return torch.sigmoid(x) 
#         return x*5 # to get values in range [0,5]

In [45]:
def get_random_batch(mat, batch_size=64):
    '''
    returns random rows of size batch_size
    '''
    rand_rows = np.random.randint(mat.shape[0], size=batch_size)
#     print(mat.shape, rand_rows)
#     print(mat[rand_rows].shape)
    return mat[rand_rows]

In [ ]:
# train = torch.autograd.Variable(torch.Tensor(train))
# augmented_train = torch.autograd.Variable(torch.Tensor(augmented_train))

In [ ]:
# del train
# del augmented_train

In [ ]:
# get_sparsity(train.cpu().numpy()), get_sparsity(augmented_train.cpu().numpy())

In [ ]:
# xx = get_random_batch(train)
# xy = get_random_batch(train)

In [ ]:
# d_my(xx, xy)

In [ ]:
# torch.sum(torch.abs(torch.abs(xx != 0).float()*xy - xy), 1)

In [ ]:
# xx > xy

In [ ]:
# # def d_my(x_r, x_g): # custom loss -todo
# #     return torch.sum(torch.abs((x_r != 0).float() * x_g - x_r), 1)/x_r.shape[1]

# def d_my(x_r, x_g): # custom loss -todo
#     return torch.sum(torch.abs(x_g - x_r), 1)/x_r.shape[1]

In [46]:
def batch_generator(corrupted, original, batch_size=64):
    rand_rows = np.random.randint(corrupted.shape[0], size=batch_size)
    return torch.Tensor(corrupted[rand_rows]).cuda().float(), torch.Tensor(original[rand_rows]).cuda().float(), rand_rows
#     return torch.from_numpy(corrupted[rand_rows]).float(), torch.from_numpy(original[rand_rows]).float()

In [47]:
# get_sparsity(X_neg), get_sparsity(y_neg)

(1.026195215919772, 1.7090908603553212)

In [ ]:
# a,b,_ = batch_generator(X_neg, (tr > 0).astype(float))

# get_sparsity(a.cpu().numpy()), get_sparsity(b.cpu().numpy()), a.shape, b.shape

In [48]:
gen_losses = []
disc_losses = []
def train_GAN(netD, netG, negative, steps_per_epoch = 100, epochs = 100):
    d_iter = 5
    g_iter = 1
    gen_iterations = 0
#     gen_losses = []
#     disc_losses = []
#     train_losses = []
    for epoch in range(epochs):
        for c in range(steps_per_epoch):
            data_iter = 100
            i = 0
            while i < 100:
                ############################
                # (1) Update D network
                ###########################
                for p in netD.parameters(): # reset requires_grad
                    p.requires_grad = True # they are set to False below in netG update
    #             d_iter = d_iter
                j = 0
                while j < d_iter*5:
                    j += 1
                    # load real data
                    i += 1
                    if negative:
                        condition, X, idxs = batch_generator(X_neg, y_neg)
    #                 X, _ = data_iter.next()
    #                 X = X.view(X.size(0), -1)
    #                 X = (X >= 0.5).float()
#                     if cuda: 
                    X = X.cuda()
                    condition = condition.cuda()
    #                 print(condition.shape, X_neg.shape, y_neg.shape)
                    real = Variable(X)

                    # generate fake data
                    noise = torch.randn(batch_size, nz)
#                     if cuda: 
                    noise = noise.cuda()
                    noisev = Variable(noise, volatile = True) # totally freeze netG
                    concated = torch.cat((noisev, condition), 1)
    #                 print(condition.shape, condition.shape, X.shape, noisev.shape, )
                    e_mask = torch.Tensor(tr[idxs]>0).cuda()
                    fake = Variable(netG(e_mask, concated).data)

                    # compute gradient, take step
                    netD.zero_grad()
    #                 concated_real = torch.cat((real, condition), 1)
    #                 print(concated_real)
                    out = netD(real, fake)
                    outputD = torch.mean(out) + lamba * out.norm()
                    stdD = torch.std(out)
                    outputD.backward(mone)
                    optimizerD.step()
#                     print('AAAAAAAAA mse:=WWWWWWWWWWWWWWWWWWWWWW')
            ############################
            # (2) Update G network
            ###########################

    #         g_iter = g_iter
            j = 0
            while j < g_iter*5:
                j += 1
                for p in netD.parameters():
                    p.requires_grad = False # to avoid computation
                netG.zero_grad()
                # load real data

                i += 1

                if negative:
                    condition, X, idxs = batch_generator(X_neg, y_neg)
                    
                X = X.cuda()
                condition = condition.cuda()
                real = Variable(X)

                # update generator
                noise = torch.randn(batch_size, nz)
                noise = noise.cuda()
                noisev = Variable(noise)
                concated_ = torch.cat((noisev, condition), 1)
                e_mask_ = torch.Tensor(tr[idxs]>0).cuda()
 
                fake = netG(e_mask_, concated_)
                out = netD(real, fake)
                outputG = torch.mean(out) + lamba * out.norm()
                stdG = torch.std(out)
                outputG.backward(one)
                optimizerG.step()
                gen_iterations += 1
#             print('AAAAAA')
            eval_loss = F.mse_loss(fake, real, reduction='mean')
#             eval_losses.append(eval_loss)
#             print('mse:', eval_loss)
#             print(outputG.item(), outputD.item())
            gen_losses.append(outputG.item())
            disc_losses.append(outputD.item())
            print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f '% (epoch, epochs, i, 100, gen_iterations, outputD.item(), outputG.item()))
    return gen_losses, disc_losses

In [ ]:
# get_sparsity(X_neg), get_sparsity(y_neg)

In [49]:
# lrD = 5e-4
# lrG = 5e-4
# batch_size = 128
# cuda = True
# epochs = 1000 #change
# seed = 1
# nz = 16
# d_iter = 5
# g_iter = 1
# lamba = 2e-4

lrD = 1e-4
lrG = 1e-4
batch_size = 64
cuda = True
epochs = 1000
device = 5
seed = 1
nz = 20
lamba = 1e-4 
# lamba = 0

In [50]:
# networks
netD_neg = NetD(negative_feedback_mask.shape[1]).cuda()
netG_neg = NetG(negative_feedback_mask.shape[1]).cuda()
print(netD_neg)
print(netG_neg)
optimizerG = optim.Adam(netG_neg.parameters(), lr=lrG, weight_decay=1e-5)
optimizerD = optim.Adam(netD_neg.parameters(), lr=lrD, weight_decay=1e-5)
one = torch.FloatTensor([1]).cuda()
mone = (-1 * one).cuda()

NetD(
  (t1): Linear(in_features=3706, out_features=1024, bias=True)
  (b1): Linear(in_features=3706, out_features=1024, bias=True)
  (fc): Linear(in_features=2048, out_features=3706, bias=True)
)
NetG(
  (netGen): Sequential(
    (0): Linear(in_features=3726, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=3706, bias=True)
    (3): Sigmoid()
  )
)


In [51]:
netG_neg.load_state_dict(torch.load('./netG_neg-1m'))
netG_neg.eval()

NetG(
  (netGen): Sequential(
    (0): Linear(in_features=3726, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=3706, bias=True)
    (3): Sigmoid()
  )
)

In [52]:
# netD_neg.train()
# netG_neg.train()
# # gen_losses, disc_losses = train_GAN(netD_neg, netG_neg, negative=True)
# train_GAN(netD_neg, netG_neg, negative=True)

In [53]:
import matplotlib.pyplot as plt

plt.plot(gen_losses)
plt.show()

<Figure size 640x480 with 1 Axes>

In [ ]:
plt.plot(disc_losses)
plt.show()

In [ ]:
# torch.save(netD_neg.state_dict(), './netD_neg-1m')
# torch.save(netG_neg.state_dict(), './netG_neg-1m')

In [54]:
import gc
gc.collect()

1283

In [55]:
# to test without train
netD_neg_test = NetD(negative_feedback_mask.shape[1]).cuda()
netG_neg_test = NetG(negative_feedback_mask.shape[1]).cuda()

netD_neg_test.eval()
netG_neg_test.eval()

NetG(
  (netGen): Sequential(
    (0): Linear(in_features=3726, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=3706, bias=True)
    (3): Sigmoid()
  )
)

In [56]:
condition, X, idxs = batch_generator(X_neg, y_neg, batch_size=256)

X = X.cuda()
condition = condition.cuda()
# real = Variable(X)

noise = torch.randn(256, nz).cuda()
e_mask = torch.Tensor(tr[idxs]>0).cuda()

concated = torch.cat((noise, condition), 1)
fake = netG_neg(e_mask, concated)

In [57]:
fake_test = netG_neg_test(e_mask, concated)

In [58]:
(fake >= 0.4).sum(), ((fake >= 0.4) * (condition==0)).sum(), condition.sum(), X.sum(), e_mask.sum()

(tensor(18172, device='cuda:0'),
 tensor(12416, device='cuda:0'),
 tensor(8360., device='cuda:0'),
 tensor(13978., device='cuda:0'),
 tensor(34686., device='cuda:0'))

In [59]:
fake = (fake > 0.4).float()

In [60]:
fake_test = (fake_test > 0.4).float()

In [61]:
(fake * condition).sum()/condition.sum(), (fake * X).sum()/X.sum()

(tensor(0.6885, device='cuda:0'), tensor(0.6916, device='cuda:0'))

In [62]:
(fake_test * condition).sum()/condition.sum(), (fake_test * X).sum()/X.sum()

(tensor(1., device='cuda:0'), tensor(1., device='cuda:0'))

In [63]:
# accuraccy on zeros (positive feedbacks)
(fake * (1-condition)*X).sum()/(X * (1 - condition)).sum()

tensor(0.6962, device='cuda:0')

In [64]:
(fake_test * (1-condition)*X).sum()/(X * (1 - condition)).sum()

tensor(1., device='cuda:0')

In [65]:
# accuraccy on zeros (positive feedbacks)
((1 - fake)*(1-condition)*(torch.from_numpy(positive_feedback_mask[idxs]).float().cuda())).cpu().numpy().sum()/((positive_feedback_mask[idxs]).astype(float)*(1-condition).cpu().numpy()).sum()

0.5892891636082673

In [66]:
((1 - fake_test)*(1-condition)*(torch.from_numpy(positive_feedback_mask[idxs]).float().cuda())).cpu().numpy().sum()/((positive_feedback_mask[idxs]).astype(float)*(1-condition).cpu().numpy()).sum()

0.0

In [67]:
del fake
del fake_test 
del e_mask

In [78]:
condition, X, idxs = batch_generator(X_neg, y_neg, batch_size=256)
# predicting on ngetavie feedbacks
condition = torch.from_numpy(X_neg).float().cuda()
X = torch.from_numpy(y_neg).cuda()


noise = torch.randn(condition.shape[0], nz).cuda()
e_mask = torch.Tensor(tr>0).cuda()

concated = torch.cat((noise, condition), 1)
fake = netG_neg(e_mask, concated)

In [79]:
fake.shape

torch.Size([6040, 3706])

In [80]:
fake = (fake > 0.4).float()

In [81]:
fake = fake.int().cpu().numpy()

In [82]:
fake = fake.astype(int)

In [83]:
fake.sum(), negative_feedback_mask.sum(), to_add_negative.sum()

(479605, 382567, 207742)

In [84]:
(fake * negative_feedback_mask).sum()/negative_feedback_mask.sum() #checking negative 

0.7044779084447953

In [85]:
(fake * to_add_negative).sum()/to_add_negative.sum() #checking negative 

0.6542971570505723

In [86]:
((1-fake) * positive_feedback_mask).sum()/positive_feedback_mask.sum() #checking positive

0.5941142264320806

In [87]:
# to_add_negative = to_add_negative.astype(int)

In [90]:
get_sparsity(to_add_negative), get_sparsity(y_neg)

(0.9280726082279318, 1.7090908603553212)

In [100]:
(y_neg*to_add_negative).sum()

184019

In [180]:
# condition, X, idxs = batch_generator(y_neg, to_add_negative, batch_size=256)
# predicting on ngetavie feedbacks
condition = torch.from_numpy(y_neg).float().cuda()
# X = torch.from_numpy(to_add_negative).cuda()


noise = torch.randn(condition.shape[0], nz).cuda()
e_mask = torch.Tensor(to_add_negative).cuda()

concated = torch.cat((noise, condition), 1)
fake = netG_neg(e_mask, concated)

In [181]:
fake = (fake > 0.5).cpu().numpy()

In [182]:
fake.sum(), to_add_negative.sum()

(125280, 207742)

In [183]:
(fake * to_add_negative * (tr==0)).sum()   # checking accuracy with autoencoder generated values

11834

In [184]:
# checking
print((y_pos * (to_add_negative*fake)).sum()/(to_add_negative*fake).sum()) 
print((y_pos * to_add_negative).sum()/to_add_negative.sum()) 
print((y_pos * fake).sum()/fake.sum()) 

0.030340038314176244
0.037700609409748634
0.030340038314176244


In [185]:
to_augment_negative = to_add_negative*fake

In [186]:
print((y_pos * to_augment_negative).sum()/to_augment_negative.sum()) 

0.030340038314176244


In [187]:
(to_augment_negative *(1-negative_feedback_mask)).sum() #new values

15635

In [188]:
to_aug = to_augment_negative *(1-negative_feedback_mask)

In [189]:
to_aug.sum()

15635

In [190]:
import nbimporter 
import matrix_factorization

In [191]:
to_add_negative.sum()

207742

In [192]:
fake.sum()

125280

In [193]:
(to_aug * (tr == 0)).sum()

11834

In [196]:
p_probs = [(tr == 1).sum()/((tr > 0) & (tr < 4)).sum(), (tr == 2).sum()/(((tr > 0) & (tr < 4))).sum(), (tr == 3).sum()/((tr > 0) & (tr < 4)).sum()]
p_probs

[0.13207882540835983, 0.25313474502505445, 0.6147864295665857]

In [197]:
augmented_train = tr + to_aug * np.random.choice(np.arange(1, 4), tr.shape, p=p_probs)

In [198]:
get_sparsity(tr), get_sparsity(augmented_train)

(4.021525859265269, 4.074393412508086)

In [ ]:
MF_SGD = matrix_factorization.ExplicitMF(tr, 40, learning='als', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 60]

# iter_array = [10]
# iter_array = [1, 2, 5, 10, 25]
MF_SGD.calculate_learning_curve(iter_array, vr, learning_rate=0.001)

Iteration: 1


In [ ]:
MF_SGD = matrix_factorization.ExplicitMF(augmented_train, 40, learning='als', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 60]

# iter_array = [10]
# iter_array = [1, 2, 5, 10, 25]
MF_SGD.calculate_learning_curve(iter_array, vr, learning_rate=0.001)